<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/Read_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source link: http://deepyeti.ucsd.edu/jianmo/amazon/index.html

In [100]:
import json
from tqdm import tqdm
import random
from collections import Counter
import pandas as pd
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive
%cd MyDrive
%cd Seminar
!ls

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/Seminar
AMAZON_FASHION.json	meta_AMAZON_FASHION.json
AMAZON_FASHION.json.gz	meta_AMAZON_FASHION.json.gz


In [ ]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz

In [ ]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_AMAZON_FASHION.json.gz

In [ ]:
# !ls

In [ ]:
# !gunzip -k AMAZON_FASHION.json.gz

In [ ]:
# !gunzip -k meta_AMAZON_FASHION.json.gz

In [ ]:
!ls

AMAZON_FASHION.json	meta_AMAZON_FASHION.json
AMAZON_FASHION.json.gz	meta_AMAZON_FASHION.json.gz


# data fast overview

In [ ]:
tweets = []
for line in open('AMAZON_FASHION.json', 'r'):
    tweets.append(json.loads(line))

In [ ]:
len(tweets) , '\n' , tweets[0]

(883636,
 '\n',
 {'asin': '7106116521',
  'overall': 5.0,
  'reviewText': 'Exactly what I needed.',
  'reviewTime': '10 20, 2014',
  'reviewerID': 'A1D4G1SNUZWQOT',
  'reviewerName': 'Tracy',
  'summary': 'perfect replacements!!',
  'unixReviewTime': 1413763200,
  'verified': True})

In [ ]:
meta_data = []
for line in open('meta_AMAZON_FASHION.json', 'r'):
    meta_data.append(json.loads(line))

In [ ]:
len(meta_data) , '\n' , meta_data[0]

(186637,
 '\n',
 {'asin': '0764443682',
  'brand': 'Group Publishing (CO)',
  'date': '8.70 inches',
  'feature': ['Product Dimensions:\n                    \n8.7 x 3.6 x 11.4 inches',
   'Shipping Weight:\n                    \n2.4 pounds'],
  'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL._US40_.jpg'],
  'imageURLHighRes': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL.jpg'],
  'rank': '13,052,976inClothing,Shoesamp;Jewelry(',
  'title': 'Slime Time Fall Fest [With CDROM and Collector Cards and Neutron Balls, Incredi-Ball and Glow Stick Necklace, Paper Fram'})

In [ ]:
t_count = i_count = hi_count = 0
tlen= []
for i in meta_data:
    if 'title' in list(i.keys()):
        t_count +=1
        tlen.append(len(i['title']))
    if 'imageURL' in list(i.keys()):
        i_count +=1
    if 'imageURLHighRes' in list(i.keys()):
        hi_count +=1

t_count , i_count , hi_count , len(meta_data)

(186632, 132017, 132017, 186637)

In [ ]:
min(tlen) , sum(tlen)/len(tlen) , max(tlen)

(1, 225.24316301598867, 424341)

In [ ]:
c = 0
for i in tqdm(range(len(tlen))):
    if tlen[i]<50:
        c+=1
        # print(c, ' --> ' , meta_data[i]['title'])
print('\n',len(tlen)-c)

100%|██████████| 186632/186632 [00:00<00:00, 2056140.90it/s]


 158048


In [ ]:
for i in meta_data[0:3]:
    print(i['rank'])
    try:
        for j in i['feature']:
            print(j)
    except:
        print('')
    print('\n ==================== \n')
    

13,052,976inClothing,Shoesamp;Jewelry(
Product Dimensions:
                    
8.7 x 3.6 x 11.4 inches
Shipping Weight:
                    
2.4 pounds


11,654,581inClothing,Shoesamp;Jewelry(



19,308,073inClothing,ShoesJewelry(
Package Dimensions:
                    
8.5 x 5.5 x 0.4 inches
Shipping Weight:
                    
6.1 ounces




# data better overview

In [ ]:
tweets[0]

{'asin': '7106116521',
 'overall': 5.0,
 'reviewText': 'Exactly what I needed.',
 'reviewTime': '10 20, 2014',
 'reviewerID': 'A1D4G1SNUZWQOT',
 'reviewerName': 'Tracy',
 'summary': 'perfect replacements!!',
 'unixReviewTime': 1413763200,
 'verified': True}

In [ ]:
meta_data[4:6]

[{'asin': '1940967805',
  'date': '5 star',
  'feature': ['Package Dimensions:\n                    \n10.5 x 6.5 x 0.2 inches',
   'Shipping Weight:\n                    \n6.4 ounces (View shipping rates and policies)'],
  'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/418HGjTDbRL._SR38,50_.jpg'],
  'imageURLHighRes': ['https://images-na.ssl-images-amazon.com/images/I/418HGjTDbRL.jpg'],
  'price': '$4.50',
  'rank': '10,558,646inClothing,Shoesamp;Jewelry(',
  'title': 'Aether & Empire #1 - 2016 First Printing Comic Book Special Edition - Rare! - Blue Juice Comics'},
 {'asin': '1942705034',
  'date': '5 star',
  'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/41XfVtdsiJL._SR38,50_.jpg',
   'https://images-na.ssl-images-amazon.com/images/I/41%2BNbtV5c5L._SR38,50_.jpg',
   'https://images-na.ssl-images-amazon.com/images/I/41DvPo37jNL._SR38,50_.jpg',
   'https://images-na.ssl-images-amazon.com/images/I/41KvxNrHleL._SR38,50_.jpg'],
  'imageURLHighRes': ['https

In [ ]:
reviewerid_dictionary = {}

for tweet in tweets:
    id = tweet['reviewerID']
    if id in reviewerid_dictionary:
        reviewerid_dictionary[id].append(tweet['asin'])
    else:
        reviewerid_dictionary[id] = [tweet['asin']]

In [ ]:
len(reviewerid_dictionary)

749233

In [ ]:
meta_data = sorted(meta_data, key=lambda d: d['asin']) 

In [ ]:
meta_data2 = meta_data[:]
meta_data = [meta_data2[0]]
asins = [meta_data2[0]['asin']]
for i in range(1,len(meta_data2)):
    if meta_data2[i]['asin'] == meta_data2[i-1]['asin']:
        if meta_data2[i]['asin'] != meta_data2[i-1]['asin']:
            print(meta_data2[i],meta_data2[i-1])
    else:
        asins.append(meta_data2[i]['asin'])
        meta_data.append(meta_data2[i])
del meta_data2

In [ ]:
asins2 = [i["asin"] for i in tweets]

In [61]:
count_user = 0
for i in reviewerid_dictionary:
    if len(reviewerid_dictionary[i]) == 0 :
        count_user += 1
asins2 = list(set(asins2))
print('cold start items=',abs(len(asins)-len(asins2)))
print('cold start users=',count_user)


cold start items= 5
cold start users= 0


* cold start items= 5

* cold start users= 0

        so for creat user cold start we should remove some users behavioural 

* 132K with imageURL

    * some has more than one image

* 158K has description with more than 50 character and white space

# save clean dataframe from users:

In [68]:
samples = random.sample(tweets,3)
for sample in samples:
    for j in sample:
        print(j,' ==> ',sample[j])
    print('====================================================================')

overall  ==>  1.0
verified  ==>  True
reviewTime  ==>  04 29, 2017
reviewerID  ==>  A3QLXKP31H7UGX
asin  ==>  B00SXU2TEO
style  ==>  {'Size:': ' Large', 'Color:': ' Multicolor'}
reviewerName  ==>  Amazon Customer
reviewText  ==>  So pretty!!  However, returned because the cut was weird in the armscye and made the shirt pull uncomfortably  when arms lifted.
summary  ==>  Wanted to like this
unixReviewTime  ==>  1493424000
overall  ==>  4.0
verified  ==>  True
reviewTime  ==>  05 25, 2016
reviewerID  ==>  AAV1QLAVQLQIB
asin  ==>  B0109W7PCE
reviewerName  ==>  Anon
reviewText  ==>  It's not too small, per se. Just tight. Needs to be worn and stretched a little
summary  ==>  Good for support, but can be annoying with how tight it is
unixReviewTime  ==>  1464134400
overall  ==>  5.0
verified  ==>  True
reviewTime  ==>  10 1, 2015
reviewerID  ==>  A12FQLIZVJUM7P
asin  ==>  B000P0X15G
style  ==>  {'Size:': ' Toddler', 'Color:': ' Aqua'}
reviewerName  ==>  An Author
reviewText  ==>  Great look

In [86]:
user_item_matrix = []

user_tweets = {}
for tweet in tqdm(tweets):
    asin = tweet['asin']
    user_id = tweet['reviewerID']
    if user_id in user_tweets:
        if asin not in user_tweets[user_id]:
            user_tweets[user_id].append([tweet])
    else:
        user_tweets[user_id] = [tweet]
    user_item_matrix.append({'user':user_id, 'item':asin, 'rate':tweet['overall']})

100%|██████████| 883636/883636 [00:03<00:00, 258857.86it/s]


In [87]:
len(user_item_matrix),len(user_tweets),len(tweets),len(meta_data)

(883636, 749233, 883636, 186194)

In [88]:
c = 0
len_ = []
for ut in tqdm(user_tweets):
    # print(ut,len(user_tweets[ut]))
    len_.append(len(user_tweets[ut]))

100%|██████████| 749233/749233 [00:00<00:00, 1139429.40it/s]


In [93]:
Counter(len_)

Counter({1: 655320,
         2: 71534,
         3: 13493,
         4: 5168,
         5: 1709,
         6: 736,
         7: 377,
         8: 510,
         9: 146,
         10: 82,
         11: 42,
         12: 27,
         13: 26,
         14: 11,
         15: 13,
         16: 9,
         17: 9,
         18: 3,
         19: 4,
         20: 1,
         21: 1,
         22: 1,
         24: 2,
         25: 1,
         26: 2,
         27: 1,
         29: 1,
         31: 1,
         32: 1,
         36: 1,
         40: 1})

about 100K with multiple buys

In [98]:
# user_tweets

In [97]:
import pickle
with open('user_tweets.p', 'wb') as fp:
    pickle.dump(user_tweets, fp, protocol=pickle.HIGHEST_PROTOCOL)
'''
with open('data.p', 'rb') as fp:
    data = pickle.load(fp)
'''

In [99]:
with open('user_item_matrix.p', 'wb') as fp:
    pickle.dump(user_item_matrix, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [101]:
with open('meta_data.p', 'wb') as fp:
    pickle.dump(meta_data, fp, protocol=pickle.HIGHEST_PROTOCOL)